In [52]:
import unicodedata
import re
import json
import os
from requests import get
from typing import Dict, List, Optional, Union, cast
from bs4 import BeautifulSoup
import pandas as pd
import requests
import nltk
from nltk.corpus import stopwords
from time import strftime
from env import github_token, github_username
import acquire

In [60]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository') 
        print('Fetching links for repositories...')
        git = 'https://github.com/'
        for repo in repositories:
            repository_links.append(repo.get('href')[1:])
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 4:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links


In [61]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

Finding max page for repositories...
Max page found: 6
Starting loop...
1 iteration
Pulling data from https://github.com/orgs/apple/repositories
Fetching links for repositories...
2 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=2
Fetching links for repositories...
3 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=3
Fetching links for repositories...
4 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=4
Fetching links for repositories...
5 iteration
Pulling data from https://github.com//orgs/apple/repositories?page=5
Fetching links for repositories...


In [62]:
repository_links

['apple/swift',
 'apple/llvm-project',
 'apple/foundationdb',
 'apple/swift-docc',
 'apple/swift-argument-parser',
 'apple/sourcekit-lsp',
 'apple/swift-protobuf',
 'apple/swift-llbuild',
 'apple/swift-syntax',
 'apple/swift-package-manager',
 'apple/swift-source-compat-suite',
 'apple/swift-markdown',
 'apple/swift-docc-symbolkit',
 'apple/swift-docc-render-artifact',
 'apple/swift-lmdb',
 'apple/swift-tools-support-core',
 'apple/swift-driver',
 'apple/indexstore-db',
 'apple/swift-stress-tester',
 'apple/swift-xcode-playground-support',
 'apple/swift-integration-tests',
 'apple/swift-cmark',
 'apple/swift-corelibs-libdispatch',
 'apple/swift-corelibs-foundation',
 'apple/swift-corelibs-xctest',
 'apple/swift-experimental-string-processing',
 'apple/swift-docker',
 'apple/swift-installer-scripts',
 'apple/swift-llbuild2',
 'apple/swift-docc-render',
 'apple/swift-evolution',
 'apple/servicetalk',
 'apple/swift-nio-ssh',
 'apple/password-manager-resources',
 'apple/swift-nio',
 'apple

In [58]:
git = 'https://github.com/'
newlist = []
for repo in repository_links:
    newlist.append(repo[1:])

In [59]:
newlist

['apple/llvm-project',
 'apple/swift-argument-parser',
 'apple/swift-docc',
 'apple/swift',
 'apple/sourcekit-lsp',
 'apple/foundationdb',
 'apple/swift-protobuf',
 'apple/swift-llbuild',
 'apple/swift-syntax',
 'apple/swift-package-manager',
 'apple/swift-source-compat-suite',
 'apple/swift-markdown',
 'apple/swift-docc-symbolkit',
 'apple/swift-docc-render-artifact',
 'apple/swift-lmdb',
 'apple/swift-tools-support-core',
 'apple/swift-driver',
 'apple/indexstore-db',
 'apple/swift-stress-tester',
 'apple/swift-xcode-playground-support',
 'apple/swift-integration-tests',
 'apple/swift-cmark',
 'apple/swift-corelibs-libdispatch',
 'apple/swift-corelibs-foundation',
 'apple/swift-corelibs-xctest',
 'apple/swift-experimental-string-processing',
 'apple/swift-docker',
 'apple/swift-installer-scripts',
 'apple/swift-llbuild2',
 'apple/swift-docc-render',
 'apple/swift-evolution',
 'apple/servicetalk',
 'apple/swift-nio-ssh',
 'apple/password-manager-resources',
 'apple/swift-nio',
 'apple

In [46]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

In [53]:
def github_api_request(url: str) -> Union[List, Dict]:
    response = requests.get(url, headers=headers)
    response_data = response.json()
    if response.status_code != 200:
        raise Exception(
            f"Error response from github api! status code: {response.status_code}, "
            f"response: {json.dumps(response_data)}"
        )
    return response_data

In [47]:
response = requests.get('https://github.com/apple/llvm-monorepo-root', headers=headers)

In [49]:
response

<Response [200]>

In [56]:
('https://github.com/apple/llvm-monorepo-root') -> Union[List, Dict]

SyntaxError: invalid syntax (1800319426.py, line 1)

In [ ]:
def get_all_repository_readme(url): # repository url
    response = get(url)
    soup = BeautifulSoup(response.text)
    repositories = soup.find_all('div', class_='org-repos')[0]
    repository_title = repositories.find_all('a', itemprop ='name codeRepository')
    title_list = [title.text.strip() for title in repository_title]
    # Get url for each repository.
    repository_links = []
    for titles in repository_title:
        repository_links.append(titles.get('href'))
    readme_list = []
    git = 'https://github.com/'
    for link in repository_links:
        repo_response = get(git + link)
        repo_soup = BeautifulSoup(repo_response.text)
        readme = repo_soup.find_all('article', itemprop='text')[0].text
        readme = readme.replace('\n',' ').replace('  ', ' ').replace('\'', '')
        readme_list.append(readme)
    dictionary = dict(zip(title_list, readme_list))
    df = pd.DataFrame(list(dictionary.items()), columns=['repo_title','readme'])
    return df

In [ ]:
url = 'https://github.com/orgs/apple/repositories'
response = get(url)

In [ ]:
soup = BeautifulSoup(response.text)

In [ ]:
repositories = soup.find_all('div', class_='org-repos')[0]

In [ ]:
repositories

In [ ]:
repositories.find_all('a', itemprop ='name codeRepository')

In [ ]:
# List of repository titles. Will use to iterate through.
repository_title = repositories.find_all('a', itemprop='name codeRepository')

In [ ]:
repository_title

In [ ]:
title_list = [title.text.strip() for title in repository_title]

In [ ]:
title_list

In [ ]:
repository_title[2].text.strip()

In [ ]:
for titles in repository_title:
    repository_links.append(titles.get('href'))

In [ ]:
# Get url for each repository.
links = repositories.find_all('a', itemprop = 'name codeRepository')

In [ ]:
repository_links = []

In [ ]:
for link in links:
    repository_links.append(link.get('href'))

In [ ]:
repository_links

In [ ]:
repository_links[0]

In [ ]:
git = 'https://github.com/'

In [ ]:
repo_response = get(git + repository_links[0])

In [ ]:
repo_response

In [ ]:
repo_soup = BeautifulSoup(repo_response.text)

In [ ]:
repo_soup.find_all('article', itemprop='text')[0].text

In [ ]:
readme = repo_soup.find_all('article', itemprop='text')[0].text

In [ ]:
readme = readme.replace('\n',' ').replace('  ', ' ').replace('\'', '')

In [ ]:
readme

In [ ]:
next_page = soup.find('a', class_='next_page').get('href')[:-1]

In [ ]:
page = 1

In [ ]:
# Use this line of code to set the next url.
next_page + str(page + 1)

In [ ]:
next_page

In [ ]:
git + next_page + max_page

In [ ]:
max_page = soup.find('div', role='navigation').text[-6]

In [ ]:
type(max_page)

In [ ]:
readme_list = []
git = 'https://github.com/'
for link in repository_links:
    repo_response = get(git + link)
    repo_soup = BeautifulSoup(repo_response.text)
    readme = repo_soup.find_all('article', itemprop='text')[0].text
    readme = readme.replace('\n',' ').replace('  ', ' ').replace('\'', '')
    readme_list.append(readme)

In [ ]:
len(readme_list)

In [ ]:
dictionary = dict(zip(title_list, readme_list))

In [ ]:
pd.DataFrame(list(dictionary.items()), columns=['repo_title','readme'])

In [ ]:
b = get_all_repository_readme('https://github.com/orgs/apple/repositories')

In [ ]:
b

In [ ]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    title_list = []
    readme_list = []
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository')
        # Create a list of the repository titles.
        repository_titles = [repo.text.strip() for repo in repositories]
        repository_titles = [e for e in repository_titles if e not in ('darwin-libpthread')]
        print("Appending titles to 'title_list'...")
        for title in repository_titles:
            title_list.append(title)
        print('Repository titles appended.')     
        print('Fetching links for repositories...')
        for repo in repositories:
            repository_links.append(repo.get('href'))
        ## Create a list of the readme files to add to a dataframe
        git = 'https://github.com/'
        #print('Fetching README files from each repository...')
        #for link in repository_links:
        #    print(f'Fetching from {link}...')
        #    # Append github to beginning of repository url
        #    repo_response = get(git + link)
        #    repo_soup = BeautifulSoup(repo_response.text)
        #    # Get the readme from the page.
        #    readme = repo_soup.find('div', id='readme').text.strip().replace('\n', ' ')
        #    readme_list.append(readme)
        #print('README files obtained.')
        next_page = soup.find('a', class_='next_page').get('href')[:-1]
        ## Use this line of code to get the url for the next page.
        if page <= 5:
            url = git + next_page + str(page + 1)
            page += 1
        else:
            return repository_links
        

In [ ]:
list_of_repositories = get_all_repository_urls('https://github.com/orgs/apple/repositories')

In [ ]:
list_of_repositories

In [ ]:
response = get('https://github.com/orgs/apple/repositories')
soup = BeautifulSoup(response.text)

In [ ]:
repositories = soup.find_all('a', itemprop='name codeRepository')

In [ ]:
list_o_repo = []
for repo in repositories:
    list_o_repo.append(repo.get('href'))

In [ ]:
list_o_repo

In [ ]:
def get_all_repository_urls(url): # repository url
    # Get max page.
    response = get(url)
    soup = BeautifulSoup(response.text)
    print('Finding max page for repositories...')
    max_page = int(soup.find('div', role='navigation').text[-6])
    print(f'Max page found: {max_page}')
    page = 1
    title_list = []
    readme_list = []
    repository_links = []
    print('Starting loop...')
    for n in range(max_page):
        print(n+1, 'iteration')
        print(f'Pulling data from {url}')
        # Reset soup.
        response = get(url)
        soup = BeautifulSoup(response.text)
        # Get all the repositories from the page.
        repositories = soup.find_all('a', itemprop='name codeRepository')
        # Create a list of the repository titles.
        repository_titles = [repo.text.strip() for repo in repositories]
        repository_titles = [e for e in repository_titles if e not in ('darwin-libpthread')]
        print("Appending titles to 'title_list'...")
        for title in repository_titles:
            title_list.append(title.text.strip())
        print('Repository titles appended.')
        # Get url for each repository.
        repository_links = []
        print('Fetching links for repositories...')
        for titles in repository_titles:
            repository_links.append(titles.get('href'))
        # Create a list of the readme files to add to a dataframe
        git = 'https://github.com/'
        #print('Fetching README files from each repository...')
        #for link in repository_links:
        #    print(f'Fetching from {link}...')
        #    # Append github to beginning of repository url
        #    repo_response = get(git + link)
        #    repo_soup = BeautifulSoup(repo_response.text)
        #    # Get the readme from the page.
        #    readme = repo_soup.find('div', id='readme').text.strip().replace('\n', ' ')
        #    readme_list.append(readme)
        #print('README files obtained.')
        next_page = soup.find('a', class_='next_page').get('href')[:-1]


In [ ]:
repository_links = get_all_repository_urls('https://github.com/orgs/apple/repositories')

In [ ]:
repository_link = get_all_repository_urls('https://github.com/orgs/apple/repositories')

In [ ]:
title_list

In [ ]:
# ccs-pyopendirectory, darwin-libplatform

In [ ]:
len(readme_list)